In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
import os
import cv2
from numpy import asarray
from PIL import Image
from functools import partial
from keras.applications import imagenet_utils

In [6]:
### Organizing classes into training, validation, and test
# print("Organizing datasets")
# masked_imgs = os.listdir('datasets/compiled/with_mask_copy')
# no_mask_imgs = os.listdir('datasets/compiled/without_mask_copy')

# i = 0
# for img in masked_imgs:
# 	if i < int(0.80*len(masked_imgs)):
# 		os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/train/mask/' + img)
# 		i += 1
# 	else:
# 		os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/validation/mask/' + img)
# 		i += 1

# j = 0
# for img in no_mask_imgs:
# 	if j < int(0.80*len(no_mask_imgs)):
# 		os.rename('datasets/compiled/without_mask_copy/' + img,'datasets/compiled/train/no-mask/' + img)
# 		j += 1
# 	else:
# 		os.rename('datasets/compiled/without_mask_copy/' + img, 'datasets/compiled/validation/no-mask/' + img)
# 		j += 1

# print("Images reorganized")

In [7]:
epochs = 1
batch_size = 16

In [8]:
# imagenet_mode_preprocessing_function = partial(imagenet_utils.preprocess_input, mode="torch")

In [20]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

validation_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

In [21]:
train_generator = train_datagen.flow_from_directory(
        batch_size=batch_size,
		directory='datasets/compiled/train/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        batch_size=batch_size,
        directory='datasets/compiled/validation/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

Found 3065 images belonging to 2 classes.
Found 768 images belonging to 2 classes.


In [22]:
### Pre-trained Model (DenseNet121 trained on Imagenet)
model = tf.keras.applications.DenseNet121(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=2)

In [23]:
# Transfer Learning
for i in model.layers:
  i.trainable = False

In [24]:
global_avg = tf.keras.layers.GlobalAveragePooling2D()(model.output)
flatten = tf.keras.layers.Flatten()(global_avg)
# drop_out = tf.keras.layers.Dropout(0.4)(flatten)
out = tf.keras.layers.Dense(2,activation='softmax')(flatten)
densenet = tf.keras.Model(inputs=[model.input],outputs=[out])

In [25]:
densenet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [26]:
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=["accuracy"])

In [27]:
history = densenet.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

192/192 [==============================] - 355s 2s/step - loss: 0.0838 - accuracy: 0.9726 - val_loss: 0.0395 - val_accuracy: 0.9909


In [28]:
densenet.save_weights('densenet121_detection_model.h5')

In [29]:
TI = Image.open('datasets/compiled/mask_test.jpeg')
test_img = np.asarray(TI)
test = test_img.copy()
test.resize(1,224,224,3)
print(test.shape)
print(densenet.predict(test))

(1, 224, 224, 3)
[[2.3569877e-10 1.0000000e+00]]


In [30]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [31]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
cv2.namedWindow("COVID Mask Detection Video Feed")
webcam = cv2.VideoCapture(0) 

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    rval, im = webcam.read()
    im=cv2.flip(im,1,1)
    
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))
 
    faces = classifier.detectMultiScale(mini)

    for f in faces:
        (x, y, w, h) = [v * size for v in f] 
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=densenet.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    if im is not None:   
        cv2.imshow('COVID Mask Detection Video Feed', im)
    key = cv2.waitKey(10)
    
    # Exit
    if key == 27: #The Esc key
        break
        
# Stop video
webcam.release()

# Close all windows
cv2.destroyAllWindows()

KeyboardInterrupt: 